In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (16,16)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# This is a matrix that maps mine board vectors to mine count vectors
def minesweepMatrix(dimensions):
    rows,cols = dimensions
    size = rows * cols
    A = np.zeros([size,size],dtype=int)
    for rA in range(size):
        for cA in range(size):
            inRow, inCol = divmod(rA,cols)
            outRow, outCol = divmod(cA,cols)
            A[rA,cA] = abs(inRow-outRow) <= 1 and abs(inCol-outCol) <= 1
    return(A)

In [4]:
# Converts a board of mines into a board of mine counts
def boardMineCounts(board):
    return(minesweepMatrix(board.shape).dot(board.flatten()).reshape(board.shape))

In [5]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    missingProbability = r.uniform(0.05,0.8)
    result = boardMineCounts(board)
    for index, x in np.ndenumerate(board):
        if x: result[index] = -1
        elif r.uniform(0, 1) < missingProbability: result[index] = -1
    return result

In [6]:
# Generates a random training batch of size at most n
def next_training_batch(n):
    batch_xs = []
    batch_ys = []
    for _ in range(n):
        board = np.random.random(dimensions) < mineProbability
        counts = boardPartialMineCounts(board)
        frees = (1 - board).flatten().astype(float)
        freesSum = sum(frees)
        if freesSum > 0:
            batch_xs.append(counts.flatten())
            batch_ys.append(frees / freesSum)
    return (np.asarray(batch_xs), np.asarray(batch_ys))

In [7]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCounts = tf.placeholder(tf.int32, [None, size], name="mineCounts")
mineCountsOneHot = tf.reshape(tf.one_hot(mineCounts+1,10), [-1, size*10])
W = tf.Variable(tf.random_normal([size*10, size], stddev=0.01), name="W")
b = tf.Variable(tf.random_normal([size], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [8]:
mineFreeAverages = tf.placeholder(tf.float32, [None, size], name="mineFreeAverages")

In [9]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=mineFreeAverages, logits=y))

In [10]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*10, size, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(b, [-1, rows, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

_ = tf.summary.scalar('accuracy', cross_entropy)

In [11]:
# Optimiser
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [12]:
# Create session and initialise or restore stuff
savePath = './saves.tf.Mines4/' + str(dimensions) + '/'
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('.', sess.graph)

In [13]:
tf.global_variables_initializer().run()

In [14]:
# Restore model?
#saver.restore(sess, savePath + "model-10000")

In [15]:
# Train
for iteration in range(10001):
    batch_xs, batch_ys = next_training_batch(100)
    if iteration % 10 == 0:
        summary, loss, _ = sess.run([merged, cross_entropy, train_step],
                                   feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
        writer.add_summary(summary, iteration)
        print('%s: Loss at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    else:
        _ = sess.run(train_step, feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
    if iteration % 1000 == 0:
        save_path = saver.save(sess, savePath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-05 20:16:53: Loss at step 0: 5.55839
Model saved in file: ./saves.tf.Mines4/(16, 16)/model-0
2017-11-05 20:17:14: Loss at step 10: 5.5504
2017-11-05 20:17:35: Loss at step 20: 5.54566
2017-11-05 20:17:56: Loss at step 30: 5.54079
2017-11-05 20:18:17: Loss at step 40: 5.53759
2017-11-05 20:18:38: Loss at step 50: 5.53386
2017-11-05 20:18:59: Loss at step 60: 5.53302
2017-11-05 20:19:20: Loss at step 70: 5.53017
2017-11-05 20:19:40: Loss at step 80: 5.52733
2017-11-05 20:20:01: Loss at step 90: 5.52527
2017-11-05 20:20:22: Loss at step 100: 5.52211
2017-11-05 20:20:43: Loss at step 110: 5.52379
2017-11-05 20:21:04: Loss at step 120: 5.52058
2017-11-05 20:21:25: Loss at step 130: 5.51919
2017-11-05 20:21:46: Loss at step 140: 5.52042
2017-11-05 20:22:07: Loss at step 150: 5.5181
2017-11-05 20:22:28: Loss at step 160: 5.51703
2017-11-05 20:22:48: Loss at step 170: 5.51741
2017-11-05 20:23:09: Loss at step 180: 5.51528
2017-11-05 20:23:30: Loss at step 190: 5.51301
2017-11-05 20:23:

2017-11-05 21:16:35: Loss at step 1710: 5.50314
2017-11-05 21:16:56: Loss at step 1720: 5.50695
2017-11-05 21:17:17: Loss at step 1730: 5.50758
2017-11-05 21:17:38: Loss at step 1740: 5.50595
2017-11-05 21:17:59: Loss at step 1750: 5.50763
2017-11-05 21:18:20: Loss at step 1760: 5.50412
2017-11-05 21:18:41: Loss at step 1770: 5.50658
2017-11-05 21:19:02: Loss at step 1780: 5.50842
2017-11-05 21:19:24: Loss at step 1790: 5.50438
2017-11-05 21:19:45: Loss at step 1800: 5.50586
2017-11-05 21:20:06: Loss at step 1810: 5.50643
2017-11-05 21:20:27: Loss at step 1820: 5.50339
2017-11-05 21:20:48: Loss at step 1830: 5.50751
2017-11-05 21:21:09: Loss at step 1840: 5.50378
2017-11-05 21:21:30: Loss at step 1850: 5.50653
2017-11-05 21:21:51: Loss at step 1860: 5.50768
2017-11-05 21:22:12: Loss at step 1870: 5.50495
2017-11-05 21:22:33: Loss at step 1880: 5.50672
2017-11-05 21:22:54: Loss at step 1890: 5.50794
2017-11-05 21:23:15: Loss at step 1900: 5.50278
2017-11-05 21:23:36: Loss at step 1910: 

KeyboardInterrupt: 

In [16]:
# Test trained model on larger batch size
batch_xs, batch_ys = next_training_batch(1000)
print(sess.run(cross_entropy, feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys}))

5.50531


In [18]:
# Run a test
batchSize = 10000
batch_xs, batch_ys = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
board = (batch_ys == 0).astype(int)
frees = [board[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

Number of errors for batch size of  10000
0


In [20]:
# Run a test
batchSize = 10000
batch_xs, batch_ys = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
worstSquares = [pred.argmin() for pred in predictions]
board = (batch_ys == 0).astype(int)
mineGuesses = [board[i][worstSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(mineGuesses))

Number of errors for batch size of  10000
9361


In [30]:
# Find boards that we failed on
batchSize = 1000
batch_xs, batch_ys = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
bestSquares = [pred.argmin() for pred in predictions]
board = (batch_ys == 0).astype(int)
mineGuesses = [board[i][bestSquares[i]] for i in range(batchSize)]
for i in range(batchSize):
    if mineGuesses[i] == 0:
        print(batch_xs[i].reshape(dimensions))
        summary = sess.run(tf.summary.image('mine_miss', tf.reshape((batch_xs[i]+1).astype(float),[-1,rows,cols,1]), 100))
        writer.add_summary(summary)

[[-1 -1 -1  0  1 -1 -1 -1 -1  0 -1 -1 -1  2 -1  0]
 [-1  0  0 -1 -1 -1 -1  2 -1 -1  1 -1 -1 -1  1  0]
 [ 0 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1  0]
 [-1 -1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1  2 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1  2 -1  2 -1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  0 -1 -1  3 -1 -1 -1  3  3  2 -1  2 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1  0 -1 -1 -1]
 [-1  1 -1 -1 -1  1 -1  3 -1  3 -1 -1 -1  0 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1  1  0 -1 -1 -1  0  0 -1  1 -1 -1]
 [-1  2  1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  1 -1]
 [ 1 -1 -1 -1 -1 -1 -1  2  3 -1 -1 -1 -1  2 -1 -1]
 [-1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  2 -1  0]
 [ 0 -1 -1  2 -1 -1  1 -1  3 -1  2  2 -1 -1 -1 -1]
 [-1 -1 -1  2 -1 -1  0 -1 -1  1 -1 -1  1 -1 -1 -1]]
[[-1 -1  1 -1 -1 -1  2  2 -1 -1  0 -1 -1 -1  2 -1]
 [-1 -1 -1 -1 -1  3 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  0 -1 -1]
 [-1  2 -1 -1 -1 -1 -1 -1 -1  

[[-1 -1 -1 -1  1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1  1]
 [-1  2  3  2 -1 -1 -1  2 -1 -1 -1  0 -1  1 -1 -1]
 [-1 -1 -1 -1 -1  0 -1  1 -1 -1  1  1  1  2 -1 -1]
 [ 0 -1  2  2 -1 -1 -1 -1  1 -1 -1 -1  2 -1 -1 -1]
 [ 0  1 -1 -1 -1 -1 -1 -1 -1  1  2 -1 -1  2 -1 -1]
 [-1 -1 -1 -1 -1  0  1 -1  2 -1 -1 -1 -1 -1  2 -1]
 [-1 -1 -1  0 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1]
 [-1 -1  3  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  1 -1  3 -1 -1 -1 -1  2 -1 -1 -1 -1]
 [-1 -1 -1  2  1 -1 -1 -1  2  2 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1]
 [ 0 -1 -1  4 -1 -1 -1 -1 -1  3 -1 -1  3 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1  4 -1 -1 -1 -1]
 [ 1 -1  2 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1  3 -1]
 [-1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1  0 -1  1  1 -1  0 -1  1 -1]]
[[-1 -1  1 -1  0  0 -1 -1 -1  1  1 -1 -1 -1 -1 -1]
 [ 0  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  2 -1]
 [ 0  2 -1 -1 -1  0 -1 -1 -1 -1  1  2 -1 -1 -1 -1]
 [-1 -1 -1  1 -1  1 -1 -1 -1 -